### 패키지

In [1]:
import numpy as np
import pandas as pd
from glob import glob

### 데이터

In [2]:
### train
train = pd.read_csv("../DATA/dataset/train/train.csv")

### test
test_dir = glob("../DATA/dataset/test/*.csv")
test_df_list = []
for i in test_dir :
    test_df_list.append(pd.read_csv(i))
test = pd.concat(test_df_list)
test["TIME"] = pd.to_datetime(test["TIME"])
test = test.sort_values(by = "TIME")

### WEATHER
weather = pd.read_csv("../DATA/WEATHER.csv")

### lunday
lunday = pd.read_csv("../DATA/lunday.csv")

In [3]:
len(train), len(test)

(109728, 33048)

### MERGE

In [6]:
lunday = lunday[['lunMonth', 'lunDay', 'lunYear']]

start = pd.to_datetime('20190101')
end = pd.to_datetime('20211231') 
df = pd.DataFrame(columns=['date'])
df['date'] = pd.date_range(start,end,freq='D')

df['Year'] = df['date'].dt.year 
df['Month'] = df['date'].dt.month 
df['Day'] = df['date'].dt.day

lun = pd.concat([df,lunday],axis=1) 
lun['date'] = lun['date'].astype('str')

# 음력 데이터 생성
df_merge_date = pd.DataFrame(pd.date_range(start='2019-01-01', end='2022-01-01',freq = '10min', name = 'datetime'))
df_merge_date.drop(157824, axis = 0, inplace = True)
df_merge_date['date'] = df_merge_date['datetime'].astype(str).str[:10]

lun_date = pd.merge(df_merge_date, lun, on = 'date', how = 'inner')
lun_date['datetime'] = lun_date['datetime'].astype(str)
lun_date

,datetime,date,Year,Month,Day,lunMonth,lunDay,lunYear
0,2019-01-01 00:00:00,2019-01-01,2019,1,1,11,26,2018
1,2019-01-01 00:10:00,2019-01-01,2019,1,1,11,26,2018
2,2019-01-01 00:20:00,2019-01-01,2019,1,1,11,26,2018
3,2019-01-01 00:30:00,2019-01-01,2019,1,1,11,26,2018
4,2019-01-01 00:40:00,2019-01-01,2019,1,1,11,26,2018
...,...,...,...,...,...,...,...,...
157819,2021-12-31 23:10:00,2021-12-31,2021,12,31,11,28,2021
157820,2021-12-31 23:20:00,2021-12-31,2021,12,31,11,28,2021
157821,2021-12-31 23:30:00,2021-12-31,2021,12,31,11,28,2021
157822,2021-12-31 23:40:00,2021-12-31,2021,12,31,11,28,2021


In [7]:
train.rename(columns = {"TIME" : "datetime"}, inplace = True)
test.rename(columns = {"TIME" : "datetime"}, inplace = True)

train['datetime'] =  pd.to_datetime(train["datetime"])
weather['datetime'] =  pd.to_datetime(weather["datetime"])
lun_date['datetime'] =  pd.to_datetime(lun_date["datetime"])

In [8]:
# train
tot_train = pd.merge(train, weather, on = 'datetime', how = 'outer')
tot_train.fillna(method = 'ffill', inplace = True)

tot_train.dropna(inplace = True)
tot_train.drop(['datetime'], axis = 1, inplace = True)

lun_train = lun_date.loc[lun_date['datetime'] < '2021-02']
tot_train = pd.concat([tot_train, lun_train], axis =1 )
tot_train.drop(['date', 'Year', 'Month','Day'], axis = 1, inplace = True)

In [9]:
# test
test_weather = weather.loc[weather['datetime'] >= '2021-03-01']

dt = pd.DataFrame(pd.date_range(start='2021-03-01', end='2022-01-01',freq = '10min', name = 'datetime'))
dt.drop(44064, axis = 0, inplace = True)


tot_test = pd.merge(dt,test, on = 'datetime', how = 'inner')
weather_df = pd.merge(dt, test_weather, on = 'datetime', how= 'outer')
weather_df.fillna(method='ffill', inplace = True)
weather_df.fillna(method='ffill', inplace = True)

tot_test = pd.merge(tot_test, weather_df, on = 'datetime', how ='inner')
lun_test = lun_date.loc[lun_date['datetime'] >= '2021-03']

tot_test = pd.merge(tot_test, lun_test, on = 'datetime', how ='inner')
tot_test.drop(['date', 'Year', 'Month','Day'], axis = 1, inplace = True)
tot_test

,datetime,ZIH120-02,PIA205B-02A_MIN,PIA205B-02A_MAX,PRESSURE-S,FI_SUM,FY_SUM,TI_MEAN,LP_TOTAL,FIA_SUM,STN-MFR-S,기온,습도,풍향,lunMonth,lunDay,lunYear
0,2021-03-01 00:00:00,0,18.192101,18.225700,102.2,22.164979,29.458889,-147.915573,504.585892,0.009701,420.318573,10.0,70.000000,225.000000,1,18,2021
1,2021-03-01 00:10:00,1,18.183001,18.225700,102.0,22.186504,29.416094,-147.939224,505.000870,0.011375,419.836670,10.0,73.333333,224.333333,1,18,2021
2,2021-03-01 00:20:00,1,18.149401,18.204300,102.0,22.131447,29.395026,-147.906815,506.194719,0.010386,417.692688,10.0,73.333333,224.333333,1,18,2021
3,2021-03-01 00:30:00,1,18.134199,18.164700,102.0,22.172283,29.521853,-147.959694,506.877647,0.008853,418.850067,10.0,73.333333,224.333333,1,18,2021
4,2021-03-01 00:40:00,1,18.094500,18.146400,102.0,22.186551,29.256739,-147.968811,505.971756,0.010260,417.646027,10.0,73.333333,224.333333,1,18,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33043,2021-12-31 22:50:00,0,17.807600,17.847300,101.3,15.917591,19.293876,-150.812553,617.455345,0.007032,529.854431,-3.0,30.000000,274.000000,11,28,2021
33044,2021-12-31 23:00:00,0,17.804600,17.841200,101.3,15.912604,19.651233,-150.836823,618.697479,0.010130,528.398926,-3.0,30.000000,274.000000,11,28,2021
33045,2021-12-31 23:10:00,0,17.798500,17.832001,101.3,15.952869,19.577471,-150.836304,618.060310,0.006596,526.316467,-3.0,30.000000,274.000000,11,28,2021
33046,2021-12-31 23:20:00,0,17.767900,17.832001,101.3,15.930380,19.566207,-150.817444,619.041084,0.007297,528.483826,-3.0,30.000000,274.000000,11,28,2021


In [10]:
tot_train.to_csv("../DATA/train.csv", index = False)
tot_test.to_csv("../DATA/test.csv", index = False)